In [13]:
import sqlite3 as lite
from pandas.io.sql import read_sql
from sqlalchemy import create_engine

engine = create_engine('sqlite://')
c = engine.connect()
conmem = c.connection
con = lite.connect('database.sqlite', isolation_level=None) #Here is the connection to <ait.sqlite> residing on disk
cur = con.cursor()
sqlx = 'SELECT name, sql FROM sqlite_master WHERE type="table"'
df = read_sql(sqlx, con, coerce_float=True, params=None) 

#Read SQLite table into a panda dataframe
df.to_sql(con=conmem, name='Table', if_exists='replace', flavor='sqlite')

/anaconda3/lib/python3.6/site-packages/pandas/io/sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


In [24]:
df

,name,sql
0,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
1,Player_Attributes,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,Player,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,Match,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,League,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,Country,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,Team,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,Team_Attributes,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [32]:
sqlx = '''
PRAGMA table_info(Player);
'''
print('Player')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])

sqlx = '''
PRAGMA table_info(Match);
'''
print('Match')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])

sqlx = '''
PRAGMA table_info(League);
'''
print('League')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])

sqlx = '''
PRAGMA table_info(Country);
'''
print('Country')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])

sqlx = '''
PRAGMA table_info(Team);
'''
print('Team')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])

sqlx = '''
PRAGMA table_info(Team_Attributes);
'''
print('Team_Attributes')
print(read_sql(sqlx, con, coerce_float=True, params=None)['name'])



Player
0                    id
1         player_api_id
2           player_name
3    player_fifa_api_id
4              birthday
5                height
6                weight
Name: name, dtype: object
Match
0                    id
1            country_id
2             league_id
3                season
4                 stage
5                  date
6          match_api_id
7      home_team_api_id
8      away_team_api_id
9        home_team_goal
10       away_team_goal
11       home_player_X1
12       home_player_X2
13       home_player_X3
14       home_player_X4
15       home_player_X5
16       home_player_X6
17       home_player_X7
18       home_player_X8
19       home_player_X9
20      home_player_X10
21      home_player_X11
22       away_player_X1
23       away_player_X2
24       away_player_X3
25       away_player_X4
26       away_player_X5
27       away_player_X6
28       away_player_X7
29       away_player_X8
             ...       
85                B365H
86                B365D
8

Which team scored the most points when playing at home?

In [53]:
sqlx = '''
SELECT Team.team_long_name, home_team_api_id, SUM(home_team_goal) as total_team_goals FROM Match
JOIN Team ON Team.team_api_id=Match.home_team_api_id
GROUP BY Match.home_team_api_id
ORDER BY total_team_goals DESC
LIMIT 5
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

,team_long_name,home_team_api_id,total_team_goals
0,Real Madrid CF,8633,505
1,FC Barcelona,8634,495
2,Celtic,9925,389
3,FC Bayern Munich,9823,382
4,PSV,8640,370


Did this team also score the most points when playing away?

In [54]:
sqlx = '''
SELECT Team.team_long_name, away_team_api_id, SUM(away_team_goal) as total_team_goals FROM Match
JOIN Team ON Team.team_api_id=Match.away_team_api_id
GROUP BY Match.away_team_api_id
ORDER BY total_team_goals DESC
LIMIT 5
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

,team_long_name,away_team_api_id,total_team_goals
0,FC Barcelona,8634,354
1,Real Madrid CF,8633,338
2,Celtic,9925,306
3,Ajax,8593,287
4,PSV,8640,282


How many matches resulted in a tie?

In [61]:
sqlx = '''
SELECT count(*) AS total_tied_matches FROM Match
WHERE away_team_goal = home_team_goal
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

,total_tied_matches
0,6596


How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [62]:
sqlx = '''
SELECT * from Player
WHERE player_name LIKE '%smith%'
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,129,160447,Adam Smith,190885,1991-04-29 00:00:00,180.34,179
1,263,30248,Alan Smith,13739,1980-10-28 00:00:00,180.34,179
2,1348,515338,Brad Smith,210389,1994-04-09 00:00:00,175.26,154
3,1513,315888,Cameron Smith,208691,1995-08-24 00:00:00,175.26,134
4,1729,34656,Chris Smith,155881,1986-03-05 00:00:00,187.96,190
5,2019,279105,Daan Smith,204389,1992-03-18 00:00:00,193.04,174
6,2416,264839,David Smith,206564,1993-03-01 00:00:00,180.34,165
7,3693,23999,Gary Naysmith,4190,1978-11-16 00:00:00,177.80,161
8,3949,91790,Gordon Smith,198372,1991-02-14 00:00:00,190.50,176
9,3958,24984,Graeme Smith,139539,1982-10-03 00:00:00,187.96,181


What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [84]:
sqlx = '''
SELECT * from Match
WHERE away_team_goal = home_team_goal
ORDER BY home_team_goal ASC
LIMIT 1
OFFSET ((SELECT count(*) AS total_tied_matches FROM Match
WHERE away_team_goal = home_team_goal)/2)
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,11168,10257,10257,2010/2011,23,2011-02-02 00:00:00,888381,8600,9857,1,...,7.5,1.5,4.2,8,1.47,4,7,1.5,4,6.5


What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [104]:
sqlx = '''
SELECT COUNT(*)/(SELECT preferred_foot, COUNT(*) from Player_Attributes
WHERE preferred_foot = 'left') from Player_Attributes
'''
read_sql(sqlx, con, coerce_float=True, params=None) 

## DOESN'T WORK

DatabaseError: Execution failed on sql '
SELECT COUNT(*)/(SELECT preferred_foot, COUNT(*) from Player_Attributes
WHERE preferred_foot = 'left') from Player_Attributes
': sub-select returns 2 columns - expected 1